# Information:
This NoteBook is here to have a clean version of what we did in the oter noteBook, it'll be the final one used, which shouuld be clean, pretty and understable

In [1]:
%matplotlib inline
import hdf5_getters
import sqlite3 as sq
import os
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression

In [2]:
local_path = 'C:/Users/Max-Pc/Downloads/millionsongsubset_full/MillionSongSubset/data'

In [3]:
def get_all_interesting_data(basedir,ext='.h5') :
    datas = []
    iteration = 0
    for root, dirs, files in os.walk(basedir):
        files = glob.glob(os.path.join(root,'*'+ext))
        for f in files:
            oneSongData = []
            h5 = hdf5_getters.open_h5_file_read(f)
            oneSongData.append( hdf5_getters.get_artist_id(h5))
            oneSongData.append(hdf5_getters.get_artist_name(h5))
            oneSongData.append(hdf5_getters.get_year(h5))
            oneSongData.append(hdf5_getters.get_danceability(h5)) #always zero
            oneSongData.append(hdf5_getters.get_duration(h5))
            oneSongData.append(hdf5_getters.get_song_hotttnesss(h5))
            oneSongData.append(hdf5_getters.get_title)
            oneSongData.append(hdf5_getters.get_tempo(h5))
            oneSongData.append(hdf5_getters.get_time_signature(h5))
            oneSongData.append(hdf5_getters.get_time_signature_confidence(h5))
            oneSongData.append(hdf5_getters.get_artist_mbtags(h5))
            oneSongData.append(hdf5_getters.get_artist_terms(h5))
            oneSongData.append(hdf5_getters.get_artist_location(h5))
            oneSongData.append(hdf5_getters.get_artist_latitude(h5))
            oneSongData.append(hdf5_getters.get_artist_longitude(h5))
            h5.close()
            iteration+=1
            if(iteration%1000== 0):
                print(iteration)
            datas.append(oneSongData)
    return datas

In [ ]:
datas = get_all_interesting_data(local_path)

In [ ]:
df = pd.DataFrame(datas,columns=['ArtistID','ArtistName','SongYear','SongDanceability','Duration','SongHotness','SongTitle','SongTempo','SongTimeSignature','SongTimeSignatureConfidence','ArtistMbtags','ArtistTerms','ArtistLocation','ArtistLatitude','ArtistLongitude'])